In [ ]:
pip install pyspark

     |████████████████████████████████| 212.3MB 69kB/s 
     |████████████████████████████████| 204kB 39.8MB/s 
  Created wheel for pyspark: filename=pyspark-3.1.1-py2.py3-none-any.whl size=212767604 sha256=b04d4f9759476078f08878eedbdb8a0f2ad052a9daa790deb4ce346ff8924cec
  Stored in directory: /root/.cache/pip/wheels/0b/90/c0/01de724414ef122bd05f056541fb6a0ecf47c7ca655f8b3c0f
Successfully built pyspark


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("Bank project").getOrCreate()

In [ ]:
df = spark.read.csv("/content/drive/MyDrive/Colab Notebooks/bank.csv",header=True,inferSchema=True)
df.printSchema()

In [ ]:
from pyspark.ml.feature import StringIndexer
indexer = StringIndexer()
indexer.setInputCol("job").setOutputCol("job_index")
df1 = indexer.fit(df).transform(df)


In [ ]:
df1.toPandas()

In [ ]:
indexer.setInputCol("marital").setOutputCol("marital_index")
df1 = indexer.fit(df1).transform(df1)
indexer.setInputCol("education").setOutputCol("education_index")
df1 = indexer.fit(df1).transform(df1)
indexer.setInputCol("default").setOutputCol("default_index")
df1 = indexer.fit(df1).transform(df1)
indexer.setInputCol("housing").setOutputCol("housing_index")
df1 = indexer.fit(df1).transform(df1)
indexer.setInputCol("loan").setOutputCol("loan_index")
df1 = indexer.fit(df1).transform(df1)
indexer.setInputCol("contact").setOutputCol("contact_index")
df1 = indexer.fit(df1).transform(df1)
indexer.setInputCol("month").setOutputCol("month_index")
df1 = indexer.fit(df1).transform(df1)
indexer.setInputCol("poutcome").setOutputCol("poutcome_index")
df1 = indexer.fit(df1).transform(df1)
indexer.setInputCol("deposit").setOutputCol("label")
df1 = indexer.fit(df1).transform(df1)

In [ ]:
df1.toPandas()

In [ ]:
feature_name=['age','balance','day','duration','campaign','pdays','previous','job_index','marital_index','education_index','default_index','housing_index','loan_index','contact_index',	'month_index'	,'poutcome_index']

In [ ]:
from pyspark.ml.feature import VectorAssembler
assembler = VectorAssembler()
assembler.setInputCols(feature_name).setOutputCol("features")
transformed_data = assembler.transform(df1)

In [ ]:
transformed_data.toPandas()

In [ ]:
(training_data, test_data) = transformed_data.randomSplit([0.8,0.2])

In [ ]:
from pyspark.ml.classification import LogisticRegression
model = LogisticRegression(featuresCol = 'features',labelCol='label', maxIter=30,threshold= 0.5)


In [ ]:
# from pyspark.ml.recommendation import ALS
# als = ALS(maxIter=5, regParam=0.01,featuresCol = 'features',labelCol='label',
#           coldStartStrategy="drop")

In [ ]:
M = model.fit(training_data)

In [ ]:
predictions = M.transform(test_data)

In [ ]:
selected = predictions.select("deposit", "prediction")
for row in selected.collect():
    deposit, prediction = row  # type: ignore
    print(
       
            deposit, prediction   # type: ignore
        
    )

In [ ]:
result = predictions.select("features", "label", "probability", "prediction") \
    .collect()
print(len(result))
for row in result:
    print("features=%s, label=%s -> prob=%s, prediction=%s"
          % (row.features, row.label, row.probability, row.prediction))

In [ ]:
predictions.toPandas()

In [ ]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

multi_evaluator = MulticlassClassificationEvaluator(labelCol = 'label', metricName = 'accuracy')
print('Logistic Regression Accuracy:', multi_evaluator.evaluate(predictions))

Logistic Regression Accuracy: 0.7959927140255009
